In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
post_master = pd.read_csv('master_latest.csv', keep_default_na=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# ONLY RUN THIS CELL IF pd.read_csv IN THE ABOVE DOES NOT WORK

import chardet

file = 'master_latest.csv'

with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(200000))
result

In [ ]:
# ONLY RUN THIS CELL IF pd.read_csv IN THE ABOVE DOES NOT WORK

file_encoding = 'utf8'        # set file_encoding to the file encoding [utf8, ascii(normally not reliable), latin1, Windows-1252, etc.]
input_fd = open('master_latest.csv', encoding=file_encoding, errors = 'backslashreplace')

post_master = pd.read_csv(input_fd, keep_default_na=False)

In [3]:
# CHANGE this if deferred records are present (default should be zero)

deferred_num = 0

In [4]:
# Check that this number agrees with its counterpart from the ORIGINAL Master

p_m = post_master.shape
p_m[0] + deferred_num

264322

In [5]:
post_master.loc[post_master.date_start == '16.11.2ß', 'date_start'] = '2020/11/16'

In [6]:
# Remove Non-standard characters

post_master.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [7]:
# Remove quotation marks in the 'comments' column

post_master.comments = post_master.comments.str.replace('"', '')

In [8]:
# Also check that this number agrees with its counterpart from the ORIGINAL Master

p_m_1 = post_master.shape
p_m_1[0] + deferred_num

264322

In [9]:
post_master.head()

,processed,who_id,dataset,prop_id,keep,duplicate_record_id,split,who_region,country_territory_area,iso,...,date_processed,uuid,original_who_code,who_id_original,flag,old_targeted,country_orig,level_orig,area_orig,date_start_orig
0,sequenced,OXCGRT_1667820,OXCGRT,,y,,n,EMRO,Afghanistan,AFG,...,4/7/2021,23de48e9-4f8b-4562-b946-59df3afe346a,7.2,OXCGRT_1667820,1,Health care workers,Afghanistan,national,,2/23/2021
1,sequenced,OXCGRT_1667821,OXCGRT,,y,,n,EMRO,Afghanistan,AFG,...,4/7/2021,a9d8d08f-3c5b-44d7-801f-ab0ab4b0cee4,7.2,OXCGRT_1667821,1,"health care workers, military",Afghanistan,national,,3/1/2021
2,sequenced,OXCGRT_1667827,OXCGRT,,y,,n,EMRO,Afghanistan,AFG,...,4/7/2021,2b045108-9789-4bad-a43d-981d5b4b32b4,7.2,OXCGRT_1667827,1,"healthcare workers,military, education",Afghanistan,national,,3/16/2021
3,sequenced,OXCGRT_2536314,OXCGRT,,y,,n,EMRO,Afghanistan,AFG,...,5/25/2021,a67f5930-d8ea-4705-b2f8-b6bf452e7260,7.2,OXCGRT_2536314,1,"age group, 40-49",Afghanistan,national,,5/4/2021
4,sequenced,OXCGRT_2989542,OXCGRT,,y,,n,EMRO,Afghanistan,AFG,...,6/7/2021,61ca2031-ef90-459e-838d-dd11c87b173a,7.2,OXCGRT_2989542,1,"healthcare workers,military,education",Afghanistan,national,,6/14/2021


In [10]:
# Test for future dates

date_start_p = post_master['date_start'].astype('datetime64[ns]')

In [11]:
date_start_p.min()

Timestamp('1900-01-13 00:00:00')

In [12]:
date_start_p.max()

Timestamp('2022-08-22 00:00:00')

In [13]:
today = pd.to_datetime("today")
today

Timestamp('2022-08-23 22:08:10.692329')

In [14]:
# Count the number of instances where the date_start column has a future date
# Typically it should not be much more than around 30.

((date_start_p > today) == True).sum()

0

In [15]:
# Check for fully blank rows. 

np.where(pd.isnull(post_master))

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
# Now check for 'shearing' of the columns (blanks are ok in the drop-down menus at this stage).

In [16]:
post_master.measure_stage.unique()

array(['new', 'modification', 'extension', 'phase-out', 're-introduction',
       'finish', '', 'extension ', 'new '], dtype=object)

In [17]:
post_master.reason_ended.unique()

array(['modification', 'new', 'extension', 'phase-out', 'finish',
       're-introduction', '', 'time limited', 'time limited ', ' '],
      dtype=object)

In [18]:
post_master.enforcement.unique()

array(['not applicable', 'recommended', 'required', 'monitored',
       'not known', 'required ', 'monitored ', '', ' not applicable'],
      dtype=object)

In [19]:
post_master.non_compliance_penalty.unique()

array(['not applicable', 'not known', 'refused entry', 'fines', 'other',
       'up to detention', 'refused exit', 'legal action',
       'arrest/detention', 'not known ', 'refused entry ', 'other ', ''],
      dtype=object)

In [20]:
post_master.trigger.unique()

array(['unknown', 'EPI', 'VOC', 'other', 'epi', 'other ', 'Epi', 'voc',
       'Other', ''], dtype=object)

In [21]:
post_master.link_live.unique()

array(['yes', 'no', 'unknown', 'Yes', '', 'YES'], dtype=object)

In [22]:
post_master.link_eng.unique()

array(['no', 'yes', 'unknown', 'No', 'NO', 'Yes', ''], dtype=object)

In [23]:
# Check that the who_region column contains no blank cells

post_master['who_region'].isnull().sum()

0

In [24]:
# Check that the admin_level column only contains allowed values

post_master.admin_level.unique()

array(['national', 'other', 'state'], dtype=object)

In [25]:
# Check that the admin_level column contains no blank cells

post_master['admin_level'].isnull().sum()

0

In [26]:
# Check the file generated. This can be used as the input for next weeks run. 

post_master.to_csv('master_Following_week.csv', index=False)

In [ ]:
# OLD SYSTEM. Remember to manually extract the rows with who codes 10 to 14 from this file and save it

# post_master.to_csv('post_master_not_10_to_14.csv', index=False)

In [27]:
# Also check that this number agrees with its counterpart from the ORIGINAL Master

post_master_count = post_master.shape[0]
post_master_count

264322

In [ ]:
# OLD SYSTEM. Note that by loading in this file, post_master now does not contain codes 10 to 14

# post_master = pd.read_csv('post_master_not_10_to_14.csv', keep_default_na=False)

In [ ]:
# OLD SYSTEM.

# post_master_not_10_to_14_count = post_master.shape[0]
# post_master_not_10_to_14_count

In [ ]:
# EURO_not_2020 = pd.read_csv('master_2020_EURO_nc.csv', keep_default_na=False)

In [ ]:
# post_master = EURO_not_2020

In [ ]:
# IGNORE this block for the time being

# Now remove rows with who_codes 10 to 14 and save them into a separate file

# code_10_to_14_list = ['14', '11', '12', '13', '14'] 

In [ ]:
# Extracted_10_to_14 = post_master[post_master.who_code.isin(code_10_to_14_list)]
# Extracted_10_to_14

In [ ]:
# post_master_not_10_to_14 = post_master[~post_master['who_code'].isin(code_10_to_14_list)]
# post_master_not_10_to_14

In [ ]:
# code_10_to_14_count = Extracted_10_to_14.shape[0]
# code_10_to_14_count

In [ ]:
# post_master_not_10_to_14_count = post_master_not_10_to_14.shape[0]
# post_master_not_10_to_14_count

In [ ]:
# check numbers - do the two data blocks sum to the original number?

# code_10_to_14_count + post_master_not_10_to_14_count == post_master_count

In [ ]:
# CHANGE the date

# Extracted_10_to_14.to_csv('10_to_14_who_codes_2022_05_24.csv', index=False)

In [ ]:
# Find the row indexes corresponding to blank who_ids '16.11.2ÃŸ'
# non_alphanumeric_row = post_master[post_master['date_start'] == '16.11.2ß'].index

In [ ]:
# CONTINUE here

In [ ]:
master_sequenced = post_master[post_master['processed']=='sequenced']

In [ ]:
# The below-mentioned remark no longer applies - instead check directly in Excel
# Is this number consistent with what is in the ORIGINAL Master?

master_sequenced.shape

In [ ]:
master_sequenced.to_csv('master_sequenced.csv', index=False)

In [ ]:
master_not_cleansed = post_master[post_master['processed']=='not_cleansed']

In [ ]:
master_not_cleansed

In [ ]:
# Is this number consistent with what is in the ORIGINAL Master?

m_n_c = master_not_cleansed.shape
m_n_c[0] + deferred_num

In [ ]:
# OLD SYSTEM

# num_not_cleansed_10_to_14 = 0  # 243 in last weeks actual run

In [ ]:
# The de-duplication is to be performed on OXFORD ONLY rows, and not EURO.

Oxford_not_cleansed = master_not_cleansed[master_not_cleansed['dataset']=='OXCGRT']

In [ ]:
Oxford_not_cleansed

In [ ]:
# master_not_cleansed[master_not_cleansed['dataset']=='OXCGRT']

In [ ]:
# For Oxford only, display a list of the repeated rows, putting aside the unique list of each row.  

duplicated_rows = Oxford_not_cleansed.duplicated(["country_territory_area", "area_covered", "who_code", "date_start","prov_measure"],keep=False)

In [ ]:
duplicated_rows

In [ ]:
Master_nc_duplicated = Oxford_not_cleansed[duplicated_rows]

In [ ]:
Master_nc_duplicated

In [ ]:
# We now convert all DUPLICATES to code 12 (not natural continuations - code 13 will be later assigned to those).
# Can NOT skip this step, even when there are no Oxford duplicates, ....... 
# ......as master_nc_duplicated_conv is required later.

In [ ]:
def summarize(group):
    # Everything inside the `group` dataframe is
    # a duplicate of its first row
    cond = group.index != group.index[0]
    group.loc[cond, 'duplicate_record_id'] = group.iloc[0]['who_id']
    group.loc[cond, 'who_code'] = '12'
    group.loc[cond, 'keep'] = 'n'
    return group



In [ ]:
master_nc_duplicated_conv = Master_nc_duplicated.groupby(["country_territory_area", "dataset", "area_covered", "who_code", "date_start","prov_measure"]).apply(summarize)

In [ ]:
master_nc_duplicated_conv

In [ ]:
# master_nc_duplicated_conv['who_code']

In [ ]:
# duplicated_rows = master_not_cleansed.duplicated(["country_territory_area", "dataset", "area_covered", "who_code", "date_start","prov_measure"],keep=False)

In [ ]:
# master_not_cleansed[duplicated_rows]

In [ ]:
# Total number of Oxford duplicates

duplicated_rows.sum()

In [ ]:
# IGNORE the following

master_not_cleansed==Master_nc_duplicated

In [ ]:
# The result of this may be wrong

Master_duplicate = master_not_cleansed[Master_nc_duplicated]
Master_duplicate

In [ ]:
# ALT Master_duplicate = master_not_cleansed[(duplicated_rows)]

In [ ]:
exclude = master_not_cleansed.isin(Master_nc_duplicated)
exclude

In [ ]:
Master_nc_duplicated

In [ ]:
Master_nc_not_duplicate = master_not_cleansed[~master_not_cleansed.isin(Master_nc_duplicated)]
Master_nc_not_duplicate

In [ ]:
Master_nc_not_duplicate = master_not_cleansed[~Master_nc_duplicated]  # ORIGINALLY [~duplicated_rows]

In [ ]:
# Continue HERE

In [ ]:
# LHS is original. RHS looks nonsense, but in fact is an emergency step for 5 Oct 2021 (and possibly beyond)
# This step may only be true provided that in an earlier step there were NO Oxford duplicates.

Master_nc_not_duplicate = master_not_cleansed 
Master_nc_not_duplicate

In [ ]:
Master_nc_not_duplicate['who_id_original'] = Master_nc_not_duplicate['who_id']

In [ ]:
Master_nc_not_duplicate['country_orig'] = Master_nc_not_duplicate['country_territory_area']

In [ ]:
Master_nc_not_duplicate['level_orig'] = Master_nc_not_duplicate['admin_level']

In [ ]:
Master_nc_not_duplicate['area_orig'] = Master_nc_not_duplicate['area_covered']

In [ ]:
Master_nc_not_duplicate['date_start_orig'] = Master_nc_not_duplicate['date_start']

In [ ]:
Master_nc_not_duplicate['date_end'] = np.where( (Master_nc_not_duplicate['dataset'] == 'CDC_ITF') & (Master_nc_not_duplicate['date_end'] == Master_nc_not_duplicate['date_start']), "" , Master_nc_not_duplicate['date_end'])

In [ ]:
# if Master_nc_not_duplicate['dataset'] == 'CDC_ITF':
#     if Master_nc_not_duplicate['date_end'] == Master_nc_not_duplicate['date_start']: 
#         Master_nc_not_duplicate['date_end'] == ""
#     else:
#         pass
# else:
#     pass

In [ ]:
cond_1 = Master_nc_not_duplicate['dataset'] == 'CDC_ITF'
cond_2 = Master_nc_not_duplicate['date_end'] == Master_nc_not_duplicate['date_start']

Master_nc_not_duplicate['date_end'] = np.where( (cond_1) & (cond_2), '', Master_nc_not_duplicate['date_end'])

In [ ]:
Master_nc_not_duplicate

In [ ]:
# Manually exammine the above-created columns in EXCEL for shear. 

Master_nc_not_duplicate.to_csv('Master_non_dup_nc.csv', index=False)

In [ ]:
# frame = Master_nc_not_duplicate

In [ ]:
Master_nc_not_duplicate['comments'] = Master_nc_not_duplicate['comments'].str.lower()

In [ ]:
Master_nc_not_duplicate['comments']

In [ ]:
# mylist = ['no change' , 'no policy change', 'no updates']

In [ ]:
# same_orig = ['no change', 'no changes', 'no further changes', 'no policy change' , 'no policy changes', 'no major policy change' ,'no updates', 'policy unchanged', 'policy remains', 'no update', 'no updates', 'no further updates', 'no new updates', 'no detected change', 'no further updates', 'still current', 'remains current', 'remains unchanged', 'no new announcements', 'no new restrictions', 'no major changes']

In [ ]:
# same_orig_df = pd.DataFrame(same_orig)

In [ ]:
# same_orig_df.to_csv('updated_list.csv', index = False)

In [ ]:
updated_start = pd.read_csv('updated_list.csv', keep_default_na=False)

In [ ]:
initial_list = updated_start['0'].tolist()

In [ ]:
initial_list

In [ ]:
# Import downloaded list of 'no change' expressions
# Additional qualifier for read_csv -  engine = 'python' 

downloaded = pd.read_excel('No_Change_Phrases.xlsx', keep_default_na=False)

In [ ]:
downloaded.dtypes

In [ ]:
downloaded = downloaded['Phrase'].str.lower()

In [ ]:
downloaded.dtypes

In [ ]:
downloaded

In [ ]:
for r in ( ('.',''), (':','') ):
    downloaded = downloaded.str.replace(*r)
    
# [x.replace(".", "") for x in downloaded_ncp]   .to_list()  .str.rstrip(".")

In [ ]:
a = 'the policy stays unchanged\xa0[comment from original change]'
A = 'the policy stays unchanged'

b = 'no policy updates [link]'
B = 'no policy updates'

c = 'as of [date], there has been no policy change'
C = 'no policy change'

# replacements = {a:A, b:B, c:C}
# downloaded_2 = downloaded.apply(replacements.get)

downloaded = downloaded.replace([a,b,c], [A,B,C])

# downloaded.Phrase.replace(['male', 'female'], [1, 0], inplace=True)

# downloaded_2 = pd.DataFrame([downloaded.get(x, x) for x in downloaded])

In [ ]:
downloaded

In [ ]:
downloaded_ncp = downloaded.drop_duplicates()

In [ ]:
downloaded_ncp.to_list()

In [ ]:
# Elements in the first list that are NOT in the second list

subtr_list = list(set(downloaded_ncp)-set(initial_list))

In [ ]:
subtr_list

In [ ]:
expanded_list = initial_list + subtr_list

In [ ]:
expanded_list

In [ ]:
expanded_list_df = pd.DataFrame(expanded_list)

In [ ]:
expanded_list_df.to_csv('updated_list.csv', index=False)

In [ ]:
pattern = '|'.join(expanded_list)

In [ ]:
def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = 'NA'
    return return_str

In [ ]:
Master_nc_not_duplicate['matched_str'] = Master_nc_not_duplicate['comments'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern))

In [ ]:
Master_nc_not_duplicate

In [ ]:
Master_nc_not_duplicate['keep'] = np.where( Master_nc_not_duplicate['matched_str']!='NA' , "n" , Master_nc_not_duplicate['keep'])

In [ ]:
Master_nc_not_duplicate['who_code'] = np.where( Master_nc_not_duplicate['matched_str']!='NA' , "13" , Master_nc_not_duplicate['who_code'])

In [ ]:
Master_nc_not_duplicate['duplicate_record_id'] = np.where(Master_nc_not_duplicate['matched_str']!='NA' , "auto13" , Master_nc_not_duplicate['duplicate_record_id'])

In [ ]:
Master_nc_not_duplicate[ ['matched_str', 'keep', 'who_code', 'duplicate_record_id', 'comments'] ]

In [ ]:
# Count the number of code 13 keep = n

(Master_nc_not_duplicate['keep']=='n').sum()

In [ ]:
# Drop the extra helper column

Master_nc_not_duplicate.drop('matched_str' , inplace=True, axis=1)

In [ ]:
Master_nc_not_duplicate

In [ ]:
# Deal with the record EURO_3855803 which has a couple of columns being 'unknown', before creating the intermediate output file.
# Possibly DROP it completely?

In [ ]:
Master_nc_not_duplicate.to_csv('Master_nc_not_duplicate.csv', index=False)

In [ ]:
# Now bring back the 'sequenced' records 

In [ ]:
code_8_list = ['8.2', '8.3', '8.4', '8.5', '8.4.1', '8.4.2']

In [ ]:
Sequenced_8 = master_sequenced[master_sequenced['who_code'].isin(code_8_list)]

In [ ]:
Sequenced_8

In [ ]:
Sequenced_other = master_sequenced[~master_sequenced['who_code'].isin(code_8_list)]

In [ ]:
Sequenced_other

In [ ]:
Sequenced_8_duplicated = Sequenced_8.duplicated(["country_territory_area", "area_covered", "who_code", "date_start", "comments"],keep='first')

In [ ]:
# This number must be noted and logged into Google Sheets

seq8_dup_count = Sequenced_8_duplicated.sum() 
seq8_dup_count

In [ ]:
# [~Sequenced_8_duplicated]

In [ ]:
# This is the number of sequenced code 8 records, after dropping the above duplicates

Sequenced_8_drop = Sequenced_8[~Sequenced_8_duplicated]

In [ ]:
Sequenced_8_drop

In [ ]:
# master_sequenced  Sequenced_8_drop, Sequenced_other

In [ ]:
Master_no_duplicates = pd.concat([Master_nc_not_duplicate, master_nc_duplicated_conv, Sequenced_8_drop, Sequenced_other])

In [ ]:
Master_no_duplicates.sort_values(["country_territory_area", "date_start"], ascending=[True, True], inplace = True)

In [ ]:
Master = Master_no_duplicates

In [ ]:
Master

In [ ]:
# Open this file in Excel, and manually remove any Oxford duplicates. 
# The counterpart with who_code 12 should be deleted.

Master.to_csv('Master_Intermediate.csv', index=False)

In [ ]:
# In this file, the Oxford duplicates were manually removed.

# Master = pd.read_csv('Master_Intermediate.csv', keep_default_na=False)

In [ ]:
# Insert the repeated column 'who_id' (previously, target column was 22)

Master.insert(23, 'who_id (repeated from column B)', Master['who_id']) 

In [ ]:
# Check for fully blank rows

np.where(pd.isnull(Master))

In [ ]:
total_num = Master.shape[0]

In [ ]:
# This number plus the number of code8 duplicates dropped, plus the deferred records should equal.....
# .....the number from the ORIGINAL Master.

total_num

In [ ]:
extracted_codes_8 = ['8.2', '8.3', '8.4', '8.5', '8.4.1', '8.4.2', '8.5']

In [ ]:
Extracted_8 = Master[Master.who_code.isin(extracted_codes_8)]

In [ ]:
Extracted_8

In [ ]:
Extracted_8_not_cleansed = Extracted_8[Extracted_8['processed']=='not_cleansed']

In [ ]:
Extracted_8_sequenced = Extracted_8[Extracted_8['processed']=='sequenced']

In [ ]:
Ex8_seq_count = Extracted_8_sequenced.shape
Ex8_seq_count

In [ ]:
Ex8_nc_count = Extracted_8_not_cleansed.shape
Ex8_nc_count

In [ ]:
# Change the date

Extracted_8_not_cleansed.to_csv('Extracted_code8_not_cleansed_2022_08_17.csv', index=False) 

In [ ]:
# IGNORE the next 3 (6?) cells and go further down

# extracted_codes_other = ['10', '11', '12', '13', '14']

In [ ]:
# Extracted_other = Master[Master.who_code.isin(extracted_codes_other)]

In [ ]:
# Extracted_other

In [ ]:
# Extracted_other = ""

In [ ]:
# Extracted_other

In [ ]:
# # Find the row indexes corresponding to blank who_ids
who_blank = Master[Master['who_id'] == ''].index

# Find the corresponding countries that correspond to these blank ids
Master['country_territory_area'].loc[who_blank]

In [ ]:
# Check for unwanted values in the 'keep' column

Master.keep.unique()

In [ ]:
Master_n = Master[Master['keep']=='n']

In [ ]:
# Note that this will be higher than its counterpart from the Original Master,
# ...because of the above conversion of additional records.

Master_n.shape[0]

In [ ]:
# Do NOT run this cell if the above cell on extracted_code_other failed.

# Check that the keep=n records do not have any unwanted codes. 
# This should agree with the above number.

# extracted_codes_2 =['10', '11', '12', '13', '14']
# Master_n['who_code'].isin(extracted_codes_2).value_counts()

In [ ]:
# Check that the who_region column only contains allowed values

Master.who_region.unique()

In [ ]:
# Check that the who_region column contains no blank cells

Master['who_region'].isnull().sum()

In [ ]:
# Check that the admin_level column only contains allowed values

Master.admin_level.unique()

In [ ]:
# Check that the admin_level column contains no blank cells

Master['admin_level'].isnull().sum()

In [ ]:
date_start = Master['date_start'].astype('datetime64[ns]')

In [ ]:
date_start.min()

In [ ]:
date_start.max()

In [ ]:
date_end = Master['date_end'].astype('datetime64[ns]')

In [ ]:
date_end.max()

In [ ]:
# Check that the following columns only contains allowed values (blanks are ok at this stage).

In [ ]:
Master.measure_stage.unique()

In [ ]:
Master.reason_ended.unique()

In [ ]:
Master.enforcement.unique()

In [ ]:
Master.non_compliance_penalty.unique()

In [ ]:
Master.link_live.unique()

In [ ]:
Master.link_eng.unique()

In [ ]:
Master.trigger.unique()

In [ ]:
# If the above original attempt at Extracted_other did NOT work, then IGNORE this cell.

# This is the Master without the 'Extracted' codes.
# It is required throughout for the 'Country List and Stats'spreadsheet 

# extracted_codes = ['8.2', '8.3', '8.4', '8.5', '8.4.1', '8.4.2', '10', '11', '12', '13', '14']
# Not_Extracted = Master[~Master.who_code.isin(extracted_codes)]
# Not_Extracted

In [ ]:
wanted_codes = ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '2.1', '2.2', '2.3', 
               '3.1', '3.1.1', '3.1.2', '3.1.3', '3.2.1', '3.2.2', 
               '4.1', '4.1.1', '4.1.2', '4.2.1', '4.2.2', '4.3.1', '4.3.2', '4.3.3', '4.3.4',
               '4.4.1', '4.4.2', '4.4.3', '4.5.1', '4.5.2', '4.5.3', '4.5.4', 
               '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9',
               '6.1', '6.2', '7.1', '7.2', '8', '8.1', '9']

Not_Extracted = Master[Master.who_code.isin(wanted_codes)]
Not_Extracted

In [ ]:
list(Not_Extracted.columns)

In [ ]:
# This should be consistent with volunteer_num below

Not_Extracted.shape[0]

In [ ]:
# All entries in the 'keep' column should be 'y' (no blanks).
# Should agree with Not_Extracted shape[0] above.

(Not_Extracted['keep']=='y').value_counts()

In [ ]:
# All the entries in 'duplicate_record_id' should be blank

(Not_Extracted['duplicate_record_id']=='').value_counts()

In [ ]:
# This step is now OBSOLETE.

# Change the date, rename the tab as 'data'.......
# ......and subsequently save as a MACRO-ENABLED Excel file.

# Not_Extracted.to_csv('master_reduced_2022_05_24.csv', index=False)

In [ ]:
# Also add in code 8 to Not_Extracted, and send to ORLAGH.

# EURO_4552187 (line 44500), OXCGRT_2990603 (line 44686)

In [ ]:
# Check that the keep=n records do not have any unwanted codes. 
# This should agree with the below number for Master_n.

wanted_codes_8 = extracted_codes_8 + wanted_codes

Int = Master_n['who_code'].isin(wanted_codes_8)
Master_n[~Int].shape[0]

In [ ]:
# This should agree with the number above. In other words, ......
# ......this file ONLY contains keep=n records.

Master_n.shape[0]

In [ ]:
Extracted_all = Master[~Master.who_code.isin(wanted_codes)]
Extracted_other = Extracted_all[~Extracted_all.who_code.isin(extracted_codes_8)]

In [ ]:
Extracted_all

In [ ]:
# Change the date

Extracted_all.to_csv('Extracted_all_2022_08_17.csv', index=False)

In [ ]:
Extracted_other.head(5)

In [ ]:
# Very Important!

Extracted_other.drop('who_id (repeated from column B)', inplace=True, axis=1)

In [ ]:
Ex_other_count = Extracted_other.shape
Ex_other_count

In [ ]:
# OLD SYSTEM Change the date

# Extracted_other.to_csv('Extracted_other_2022_05_24.csv', index=False) 

In [ ]:
extracted_num = Extracted_other.shape[0] + Extracted_8.shape[0]

In [ ]:
volunteer_num = total_num - extracted_num
volunteer_num

In [ ]:
vol_not_cleansed = Not_Extracted[Not_Extracted['processed']=='not_cleansed']

In [ ]:
# List the number of new records per country.

vol_countries = vol_not_cleansed.groupby(['country_territory_area'])[['processed']].count()
vol_df = pd.DataFrame(vol_countries).reset_index()

In [ ]:
# Convert the 'GroupBy' dataframe to a list

vol_list = vol_df['country_territory_area'].tolist()

In [ ]:
# Construct the Volunteer Master

Vol_File = Not_Extracted[Not_Extracted['country_territory_area'].isin(vol_list)]
Vol_File = Vol_File.sort_values('country_territory_area', ascending=True)
Vol_File

In [ ]:
# This is the number of rows to be cleaned by the volunteers

(Vol_File['processed'] == 'not_cleansed').sum()

In [ ]:
# Also need to find the number of volunteer rows in codes 1 to 7. 

codes_1to7 = ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '2.1', '2.2', '2.3', 
               '3.1', '3.1.1', '3.1.2', '3.1.3', '3.2.1', '3.2.2', 
               '4.1', '4.1.1', '4.1.2', '4.2.1', '4.2.2', '4.3.1', '4.3.2', '4.3.3', '4.3.4',
               '4.4.1', '4.4.2', '4.4.3', '4.5.1', '4.5.2', '4.5.3', '4.5.4', 
               '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9',
               '6.1', '6.2', '7.1', '7.2']

In [ ]:
Vol_1to7 = Vol_File[Vol_File.who_code.isin(codes_1to7)]  
(Vol_1to7['processed'] == 'not_cleansed').sum()

In [ ]:
# For the Volunteer file, find out how many records there are by each who_code

Vol_not_cleansed = Vol_File[Vol_File['processed'] == 'not_cleansed']
Vol_not_cleansed.groupby(['who_code'])[['processed']].count()

In [ ]:
# Ensure that the 'queries_comments' column is blank.

Vol_File['queries_comments'] = ''

In [ ]:
vol_count = Vol_File.shape
vol_count

In [ ]:
# This step is currently SUSPENDED while the cleaning of sequenced EURO records is taking place.

# Change the date, rename the tab as 'data'.......
# ......and subsequently save as a MACRO-ENABLED Excel file.

# Vol_File.to_csv('Volunteer_Rows_2022_05_24.csv', index=False)

In [ ]:
# Construct the Residual Master containing NO new records
# This will be combined with the 'Extracted_other' file. 

No_New_Countries = Not_Extracted[~Not_Extracted['country_territory_area'].isin(vol_list)]
No_New_Countries = No_New_Countries.sort_values('country_territory_area', ascending=True)
No_New_Countries

In [ ]:
# Count the number of countries with NO new records

len(No_New_Countries.groupby(['country_territory_area']))

In [ ]:
no_new_count = No_New_Countries.shape
no_new_count

In [ ]:
# OLD SYSTEM Change the date

# No_New_Countries.to_csv('No_New_Records_2022_05_24.csv', index=False)

In [ ]:
# Recap of the number of rows in the ORIGINAL master

post_master_count = post_master.shape
post_master_count

In [ ]:
seq8_dup_count

In [ ]:
# Numbers check (OLD SYSTEM) - if there is disagreement, then it will say (False, False) or only (False).

seq8_dup_count + Ex8_nc_count + Ex8_seq_count + Ex_other_count + vol_count + no_new_count == post_master_count

In [ ]:
# IGNORE this cell

# cond_deu = No_New_Countries['country_territory_area']=='Germany'                                       
# No_New_Countries_deu = No_New_Countries[(cond_deu)]
# No_New_Countries_deu

In [ ]:
# IGNORE this cell

# cond_uk = No_New_Countries['country_territory_area']=='United Kingdom Of Great Britain And Northern Ireland'
# No_New_Countries_uk = No_New_Countries[(cond_uk)]
                                       
# No_New_Countries_EURO = pd.concat([No_New_Countries_deu, No_New_Countries_uk], axis=0)  
# No_New_Countries_EURO = No_New_Countries_uk.append(No_New_Countries_deu, ignore_index=True)
    
# No_New_Countries_EURO = No_New_Countries[No_New_Countries['who_region']=='EURO']
# No_New_Countries_EURO

In [ ]:
# This is an extra procedure introduced to help with preparing next weeks mistress.

No_New_Countries_EURO = No_New_Countries[No_New_Countries['who_region']=='EURO']
No_New_Countries_EURO

In [ ]:
# REACTIVATED on 12 July 2022 ( Originally de-activated on 28 June 2022) 

# This is PURELY for Macro Splitting. Do NOT upload to the Task folder on MS Teams.
# CHANGE date. 

No_New_Countries_EURO.to_csv('No_New_Countries_EURO_Not_for_MS_Teams_2022_08_17.csv', index=False) 

In [ ]:
# This is PURELY for Macro Splitting. Do NOT upload to the Task folder on MS Teams.
# CHANGE date. 

Vol_File.to_csv('Vol_rows_Not_for_MS_Teams_2022_08_17.csv', index=False)

In [ ]:
# This is an extra procedure introduced to help with preparing next weeks mistress.

# No_New_Countries_EURO = No_New_Countries[No_New_Countries['who_region']=='EURO']
# No_New_Countries_EURO

In [ ]:
# Vol_File_with_EURO = Vol_File

Vol_File_with_EURO = Vol_File.append(No_New_Countries_EURO, ignore_index=True)

In [ ]:
vol_euro_count = Vol_File_with_EURO.shape
vol_euro_count

In [ ]:
# This file operates the 'Country List and Stats' spreadsheet

# Change the date, rename the tab as 'data'.......
# ......and subsequently save as a MACRO-ENABLED Excel file.

Vol_File_with_EURO.to_csv('Master_VolunteerEURO_2022_08_17.csv', index=False)

In [ ]:
# The only EURO countries from no_new_records that are being cast aside are UK and Germany.

# cond_not_uk = No_New_Countries['country_territory_area']!='United Kingdom Of Great Britain And Northern Ireland'

# No_New_Countries_not_euro = No_New_Countries[(cond_not_uk)]  # Check this.

In [ ]:
No_New_Countries_not_euro = No_New_Countries[No_New_Countries['who_region']!='EURO']
No_New_Countries_not_euro

In [ ]:
No_New_Countries_not_euro.drop('who_id (repeated from column B)', inplace=True, axis=1)

In [ ]:
# This is NOT to be added to the 'Task' folder, as it will later be concatenated.

# No_New_Countries_not_euro.to_csv('No_New_Not_Euro_2022_06_07.csv', index=False)

In [ ]:
Non_Vol = Extracted_other.append(No_New_Countries_not_euro, ignore_index=True)

In [ ]:
Extracted_8_sequenced.drop('who_id (repeated from column B)', inplace=True, axis=1)

In [ ]:
# This dataset is to be concatenated with the incomplete mistress prepared the following week
# Manaully concatenate it in EXCEL, perform a Sort Order, and then a final test on mistress.
# Re-upload the full mistress to Slack for Lukas.

Non_Vol_with_8_seq = Non_Vol.append(Extracted_8_sequenced, ignore_index=True)

In [ ]:
Non_Vol_count = Non_Vol_with_8_seq.shape
Non_Vol_count

In [ ]:
# Change the date

Non_Vol_with_8_seq.to_csv('Non_Volunteer_2022_08_17.csv', index=False)

In [ ]:
seq8_dup_count

In [ ]:
Ex8_nc_count[0]

In [ ]:
vol_euro_count[0]

In [ ]:
Non_Vol_count[0]

In [ ]:
post_master_count[0]

In [ ]:
# Numbers check - if there is disagreement, then it will say (False, False) or only (False).

seq8_dup_count + Ex8_nc_count[0] + vol_euro_count[0] + Non_Vol_count[0] == post_master_count[0]

In [ ]:
# May not need the below, as we already have a standalone file for EURO_Non-volunteer.

In [ ]:
# Find out the names of those EURO countries without new records

import pandas as pd

df = pd.read_excel('Country_List.xlsx', sheetname=0) # can also index sheet by name or fetch all sheets
mylist_new = df['Not Cleansed'].dropna().unique().tolist()
mylist_seq = df['Seq and Not Cleansed'].tolist()

In [ ]:
len(mylist_seq)

In [ ]:
len(mylist_new)

In [ ]:
# Elements in the first list that are NOT in the second list

non_vol_euro_list = list(set(mylist_seq)-set(mylist_new))
sorted(non_vol_euro_list)

In [ ]:
len(mylist_seq) - len(mylist_new)